<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_06_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 06 NLP

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [22]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
path_otzv = "/content/gdrive/My Drive/Colab Notebooks/NLP/отзывы за лето.xls"
data = pd.read_excel(path_otzv)
data.tail(3)

,Rating,Content,Date
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [8]:
from datetime import datetime

# data["Year"] = [datetime.strptime(dt, '%Y-%m-%d').year for dt in data["Date"].values]
data["Month"] = [datetime.strptime(dt, '%Y-%m-%d').month for dt in data["Date"].values]
data["Day"] = [datetime.strptime(dt, '%Y-%m-%d').day for dt in data["Date"].values]
data.head()

,Rating,Content,Date,Month,Day
0,5,It just works!,2017-08-14,8,14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,8,14
2,5,Отлично все,2017-08-14,8,14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,8,14
4,5,"Очень удобно, работает быстро.",2017-08-14,8,14


## Задание

Провести сравнение RNN, LSTM, GRU на датасете отзывов

In [9]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=66e8b01863e4b47f9a8e82264346eed12601882170b707d1f83087c4cf891f5e
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [10]:
pip install pymorphy2

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 8.2MB 3.2MB/s 


In [11]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [12]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [13]:
data['Content_prep'] = data['Content'].apply(preprocess_text)
data.tail()

,Rating,Content,Date,Month,Day,Content_prep
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,6,1,шляпас роот право бесполезный прогаразрабыв ох...
20655,5,Ок,2017-06-01,6,1,около
20656,4,Доволен,2017-06-01,6,1,довольный
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,6,1,песопаснастя рута нужын
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01,6,1,сбербанк бомбовый компания сегодняшний лидер б...


In [14]:
train_corpus = " ".join(data['Content_prep'])

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens_tot = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
tokens_filtered = [word for word in tokens_tot if word.isalnum()]

In [17]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

In [18]:
vocabulary = {v: i+100  for i, v in enumerate(dist) if dist[v]>1}

In [19]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
data_embeded = np.asarray([text_to_sequence(text, 8) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack((data_embeded, (data['Month'].values+50).reshape(data['Month'].values.shape[0], 1)))
data_embeded = np.hstack((data_embeded, (data['Day'].values).reshape(data['Day'].values.shape[0], 1)))
data_embeded

array([[   0,    0,    0, ...,  102,   58,   14],
       [ 107,  108,  109, ...,  115,   58,   14],
       [   0,    0,    0, ...,  116,   58,   14],
       ...,
       [   0,    0,    0, ...,  274,   56,    1],
       [   0,    0,    0, ...,  525,   56,    1],
       [   0,  182, 9232, ..., 3814,   56,    1]])

In [25]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [26]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 10, 1)

In [27]:
def to_one_hot(Y):
    n_col = np.amax(Y)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [28]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

## RNN

## LSTM

## GRU